In [1]:
from vllm import LLM, SamplingParams

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = SamplingParams(temperature=0.8, top_p=0.95)

In [3]:
llm = LLM(model="Qwen/Qwen2.5-1.5B-Instruct")

2025-04-23 18:02:17,614 - modelscope - WARNING - Using branch: master as version is unstable, use with caution
2025-04-23 18:02:17,907 - modelscope - INFO - Got 9 files, start to download ...


Processing 9 items:   0%|          | 0.00/9.00 [00:00<?, ?it/s]

2025-04-23 18:02:19,116 - modelscope - INFO - Download model 'Qwen/Qwen2.5-1.5B-Instruct' successfully.
2025-04-23 18:02:19,118 - modelscope - INFO - Creating symbolic link [/mnt/workspace/.cache/modelscope/models/Qwen/Qwen2.5-1.5B-Instruct].


INFO 04-23 18:02:19 llm_engine.py:176] Initializing an LLM engine (v0.5.3) with config: model='Qwen/Qwen2.5-1.5B-Instruct', speculative_config=None, tokenizer='Qwen/Qwen2.5-1.5B-Instruct', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None), seed=0, served_model_name=Qwen/Qwen2.5-1.5B-Instruct, use_v2_block_manager=False, enable_prefix_caching=False)


2025-04-23 18:02:19,924 - modelscope - INFO - Target directory already exists, skipping creation.


...

2025-04-23 18:02:32,332 - modelscope - INFO - Got 1 files, start to download ...


Processing 1 items:   0%|          | 0.00/1.00 [00:00<?, ?it/s]

2025-04-23 18:02:56,294 - modelscope - INFO - Download model 'Qwen/Qwen2.5-1.5B-Instruct' successfully.
2025-04-23 18:02:56,295 - modelscope - INFO - Target directory already exists, skipping creation.


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 04-23 18:03:06 model_runner.py:692] Loading model weights took 2.8875 GB
INFO 04-23 18:03:08 gpu_executor.py:102] # GPU blocks: 34538, # CPU blocks: 9362
INFO 04-23 18:03:10 model_runner.py:980] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 04-23 18:03:10 model_runner.py:984] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 04-23 18:03:20 model_runner.py:1181] Graph capturing finished in 9 secs.


In [4]:
outputs = llm.generate(prompts, sampling_params)

for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")

Processed prompts: 100%|██████████| 4/4 [00:00<00:00, 13.84it/s, est. speed input: 76.15 toks/s, output: 221.51 toks/s]

Prompt: 'Hello, my name is', Generated text: " Kailin. I'm a 17 year old girl from Washington."
Prompt: 'The president of the United States is', Generated text: " the nation's head of state, head of government, and commander-in-chief of"
Prompt: 'The capital of France is', Generated text: ' also the capital of Europe and a major financial center, which of the following cities'
Prompt: 'The future of AI is', Generated text: ' about making it work with the world, not against it.\nThis is the thinking'


# Online vllm

In [1]:
!curl http://localhost:8000/v1/models

{"object":"list","data":[{"id":"Qwen/Qwen2.5-1.5B-Instruct","object":"model","created":1745402823,"owned_by":"vllm","root":"Qwen/Qwen2.5-1.5B-Instruct","parent":null,"max_model_len":32768,"permission":[{"id":"modelperm-c4eabcf3d2c24faca18b77e329acd4e6","object":"model_permission","created":1745402823,"allow_create_engine":false,"allow_sampling":true,"allow_logprobs":true,"allow_search_indices":false,"allow_view":true,"allow_fine_tuning":false,"organization":"*","group":null,"is_blocking":false}]}]}

In [10]:
from openai import OpenAI

# Modify OpenAI's API key and API base to use vLLM's API server.
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"
client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)
completion = client.completions.create(model="Qwen/Qwen2.5-1.5B-Instruct",
                                      prompt="San Francisco is a",
                                      temperature=0)
print("Completion result:", completion)

Completion result: Completion(id='cmpl-10382aa66f2b43af9255ec99772d17b3', choices=[CompletionChoice(finish_reason='length', index=0, logprobs=None, text=' city in the state of California, in the United States. It is the capital', stop_reason=None)], created=1745403069, model='Qwen/Qwen2.5-1.5B-Instruct', object='text_completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=16, prompt_tokens=4, total_tokens=20, completion_tokens_details=None, prompt_tokens_details=None))


In [12]:
from openai import OpenAI
# Set OpenAI's API key and API base to use vLLM's API server.
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

chat_response = client.chat.completions.create(
    model="Qwen/Qwen2.5-1.5B-Instruct",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Tell me a joke."},
    ]
)
print("Chat response:", chat_response)

Chat response: ChatCompletion(id='chat-c3c9853a08e6459388c8cb2aeea6d7d5', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="Sure, here's a joke for you:\n\nWhy couldn't the bicycle stand up by itself?\n\nBecause it was two-tired!", refusal=None, role='assistant', annotations=None, audio=None, function_call=None, tool_calls=[]), stop_reason=None)], created=1745403139, model='Qwen/Qwen2.5-1.5B-Instruct', object='chat.completion', service_tier=None, system_fingerprint=None, usage=CompletionUsage(completion_tokens=27, prompt_tokens=24, total_tokens=51, completion_tokens_details=None, prompt_tokens_details=None))
